In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-midterm-p2/train.csv
/kaggle/input/2022-ai-midterm-p2/test.csv
/kaggle/input/2022-ai-midterm-p2/submit_sample.csv


In [2]:
#랜덤시드 고정
import random
import torch

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022-ai-midterm-p2/submit_sample.csv')
train = pd.read_csv('/kaggle/input/2022-ai-midterm-p2/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-midterm-p2/test.csv')

In [4]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
#데이터 정보 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       404 non-null    int64  
 1   CRIM     404 non-null    float64
 2   ZN       404 non-null    float64
 3   INDUS    404 non-null    float64
 4   CHAS     404 non-null    int64  
 5   NOX      404 non-null    float64
 6   RM       404 non-null    float64
 7   AGE      404 non-null    float64
 8   DIS      404 non-null    float64
 9   RAD      404 non-null    int64  
 10  TAX      404 non-null    int64  
 11  PTRATIO  404 non-null    float64
 12  B        404 non-null    float64
 13  LSTST    404 non-null    float64
 14  MEDV     404 non-null    float64
dtypes: float64(11), int64(4)
memory usage: 47.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       10

In [6]:
train

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTST,MEDV
0,0,0.14150,0.0,6.91,0,0.448,6.169,6.6,5.7209,3,233,17.9,383.37,5.81,25.3
1,1,0.15445,25.0,5.13,0,0.453,6.145,29.2,7.8148,8,284,19.7,390.68,6.86,23.3
2,2,16.81180,0.0,18.10,0,0.700,5.277,98.1,1.4261,24,666,20.2,396.90,30.81,7.2
3,3,0.05646,0.0,12.83,0,0.437,6.232,53.7,5.0141,5,398,18.7,386.40,12.34,21.2
4,4,8.79212,0.0,18.10,0,0.584,5.565,70.6,2.0635,24,666,20.2,3.65,17.16,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,399,0.03548,80.0,3.64,0,0.392,5.876,19.1,9.2203,1,315,16.4,395.18,9.25,20.9
400,400,0.09164,0.0,10.81,0,0.413,6.065,7.8,5.2873,4,305,19.2,390.91,5.52,22.8
401,401,5.87205,0.0,18.10,0,0.693,6.405,96.0,1.6768,24,666,20.2,396.90,19.37,12.5
402,402,0.33045,0.0,6.20,0,0.507,6.086,61.5,3.6519,8,307,17.4,376.75,10.88,24.0


In [7]:
#데이터 전처리
xtrain = train.drop(['ID','MEDV'],axis=1)
ytrain = train['MEDV']
xtest = test.drop(['ID'],axis=1)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

xtrain_std = sc.fit_transform(xtrain)
xtest_std = sc.transform(xtest)

In [8]:
# X_train = torch.FloatTensor(np.array(xtrain)).to(device)
# y_train = torch.FloatTensor(np.array(ytrain)).to(device)
# X_test = torch.FloatTensor(np.array(xtest)).to(device)
X_train = torch.FloatTensor(np.array(xtrain_std)).to(device)
y_train = torch.FloatTensor(np.array(ytrain)).to(device)
X_test = torch.FloatTensor(np.array(xtest_std)).to(device)

In [9]:
#모델 빌드에 필요한 설정값 확인
print(X_train.shape)
print(y_train.shape)

torch.Size([404, 13])
torch.Size([404])


In [10]:
#모델 빌드
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(13, 512,bias=True)
        self.linear2 = nn.Linear(512, 512,bias=True)
        self.linear3 = nn.Linear(512, 512,bias=True)
        self.linear4 = nn.Linear(512, 1,bias=True)
        
        self.active = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        nn.init.xavier_normal_(self.linear4.weight)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear4(x)
        
        return x

In [11]:
#모델 정의
model1 = DNN().to(device)

In [12]:
import torch.optim as optim

#Optimizer 정의
optimizer = optim.Adam(model1.parameters(), lr=1e-3)

#Loss 정의 (회귀문제이므로 MSELoss)
loss = nn.MSELoss()

In [13]:
#모델 학습
epochs = 10000
for epoch in range(epochs + 1):
    
    optimizer.zero_grad()
    
    hypothesis = model1(X_train)
    
    cost = loss(hypothesis ,y_train.unsqueeze(1))
    
    cost.backward()
    
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(epoch, cost.item())

0 582.80126953125
1000 3.1805286407470703
2000 2.1463098526000977
3000 1.8916264772415161
4000 1.467797040939331
5000 1.4483104944229126
6000 1.3571479320526123
7000 0.996638298034668
8000 0.9676797389984131
9000 1.002044916152954
10000 0.934146523475647


In [14]:
#학습한 모델을 이용해서 Test 데이터 예측
with torch.no_grad():
    model1.eval()
    predict = model1(X_test)

In [15]:
#예측한 값을 csv로 변환
sample['MEDV'] = predict.cpu().detach().numpy()
sample.to_csv('submit.csv',index=False)